Before you turn this lab in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
DRIVER = "Srini"
NAVIGATOR = "Chip"

# Autoencoder Lab

Welcome to the autoencoder lab! By the end of this lab you will have

- Created and tuned an autoencoder in Keras

*The code in the lab was based on [this blogpost](https://blog.keras.io/building-autoencoders-in-keras.html)*.

# Auto-Grader Variables

In [2]:
ENCODING_DIM = 32

def passed(): print('✅')

# Load MNIST

The following code loads MNIST and normalizes it and flattens it.

In [3]:
from IPython.display import display
from keras.datasets import mnist
from keras.preprocessing.image import array_to_img
import numpy as np

[X_train, _], [X_test, _] = mnist.load_data()
X_train = np.array([x.flatten() for x in X_train.astype('float32')/255.])
X_test = np.array([x.flatten() for x in X_test.astype('float32')/255.])

for x in X_train[:5]:
    img = array_to_img(x.reshape(28, 28, 1))
    display(img)

Using TensorFlow backend.


# Task

- Define an encoder which maps a flattened MNIST image to a vector of size `ENCODING_DIM`

# Requirements

- Use a keras sequential model
- Do not compile your model!

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.regularizers import l1

ENCODING_DIM = 32

encoder = Sequential(name='encoder')
encoder.add(Dense(units=ENCODING_DIM, activity_regularizer=l1(1e-5), input_shape=[784]))
encoder.add(Activation('sigmoid'))

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_1 (Activation)    (None, 32)                0         
Total params: 25,120.0
Trainable params: 25,120.0
Non-trainable params: 0.0
_________________________________________________________________


# Encoder Tests

In [5]:
import keras

assert type(encoder) == keras.models.Sequential
assert encoder.input_shape == (None, 784)
assert encoder.output_shape == (None, ENCODING_DIM)
assert not hasattr(encoder, 'optimizer')
encoder.predict(X_train[:1])

passed()

✅


# Task

- Define a decoder which maps a compressed version of an image back to a flattened image

# Requirements

- Use a keras sequential model
- Do not compile your model!

In [6]:
decoder = Sequential(name='decoder')

decoder.add(Dense(units=64, activation='relu', input_shape=[ENCODING_DIM]))
decoder.add(Dense(units=128, activation='relu'))
decoder.add(Dense(units=784, activation='sigmoid'))

decoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 784)               101136    
Total params: 111,568.0
Trainable params: 111,568
Non-trainable params: 0.0
_________________________________________________________________


# Decoder Tests

In [7]:
import keras

assert type(decoder) == keras.models.Sequential
assert decoder.input_shape == (None, ENCODING_DIM)
assert decoder.output_shape == (None, 784)
assert not hasattr(encoder, 'optimizer')
decoder.predict(encoder.predict(X_train[:1]))

passed()

✅


# Task

- Define an autoencoder by hooking together `encoder` and `decoder` via the keras functional API

# Requirements

- Compile the model with a binary crossentropy loss on a per-pixel basis

In [8]:
from keras.models import Model
from keras.layers import Input

img = Input(shape=[784])
hidden = encoder(img)
original = decoder(hidden)

autoencoder = Model(inputs=img, outputs=original, name='autoencoder')
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
encoder (Sequential)         (None, 32)                25120     
_________________________________________________________________
decoder (Sequential)         (None, 784)               111568    
Total params: 136,688.0
Trainable params: 136,688.0
Non-trainable params: 0.0
_________________________________________________________________


# Autoencoder Tests

In [9]:
import numpy as np

assert type(autoencoder) == keras.engine.training.Model
assert autoencoder.input_shape == encoder.input_shape
assert autoencoder.output_shape == decoder.output_shape
assert hasattr(autoencoder, 'optimizer')
X = X_train[:1]
X_encoded = encoder.predict(X)
assert np.all(decoder.predict(X_encoded) == autoencoder.predict(X))
assert len(autoencoder.layers) == 3
assert type(autoencoder.layers[1]) == keras.models.Sequential
assert type(autoencoder.layers[2]) == keras.models.Sequential
assert type(autoencoder.layers[0]) == keras.engine.topology.InputLayer

passed()

✅


# Task

- Optimize your autoencoder on the training data

In [10]:
autoencoder.fit(X_train, X_train, epochs=5, validation_data=(X_test, X_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 15s - loss: 0.2639 - val_loss: 0.2409

# Task

- Visualize the performance of your autoencoder

# Requirements

- Convert each image in `X_test` to a `PIL` image and save it into a python list called `test_imgs`
- Run each image in `X_test` through your autoencoder then convert it back into a `PIL` image and save it into a python list called `reconstructed_imgs`

In [11]:
X_encoded = encoder.predict(X_test)
X_reconstructed = decoder.predict(X_encoded)

print(X_encoded.shape); print(X_reconstructed.shape)

test_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_test]
reconstructed_imgs = [array_to_img(x.reshape(28, 28, 1)) for x in X_reconstructed]

idxs = np.random.choice(len(X_test), size=5)
for idx in idxs:
    real_img, reconstructed_img = test_imgs[idx], reconstructed_imgs[idx]
    display(real_img); display(reconstructed_img)

(10000, 32)
(10000, 784)


# Autoencoder Visualization Tests

In [12]:
import PIL

assert len(test_imgs) == len(X_test)
for test_img, reconstructed_img in zip(test_imgs, reconstructed_imgs):
    assert type(test_img) == PIL.Image.Image
    assert type(reconstructed_img) == PIL.Image.Image
    assert test_img.size == (28, 28)
    assert reconstructed_img.size == (28, 28)
    
passed()

✅


# Challenge Activities

- Add a sparsity constraint to the encoder
- Make the encoder deeper
- Make the decoder deeper
- Add noise to your data samples